Here we'll show some of the inner workings of the algorithm to compute convex hulls.
First, we'll generate a random set of points as our input data using the random number generation routines in numpy.
To make sure that this demo gives the same results every time, we'll explicitly seed the RNG with the number 1729, which as we all know is [a rather dull one](https://en.wikipedia.org/wiki/1729_(number)).

In [ ]:
import numpy as np
from numpy import random as random
rng = random.default_rng(seed=1729)
num_points = 120
X = rng.uniform(size=(num_points, 2))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.scatter(X[:, 0], X[:, 1]);

To start calculating the convex hull, we'll create a state machine object which we'll call `hull_machine`.
This state machine stores the current value of the hull geometry in the member `geometry` as well as some ancillary data for the algorithm, like a queue of which edges to inspect next.

In [ ]:
import zmsh
hull_machine = zmsh.ConvexHullMachine(X)
geometry = hull_machine.geometry

The two methods of the hull machine that we care about are `step` and `is_done`.
The `step` method will inspect an edge of the hull to see whether there's a hull point across from it.
If there is a hull point across that edge, the edge will be split in two along the new hull point.
Then any points inside the triangle formed by the old edge and the two new edges will be filtered out as candidate hull points.
If not, then nothing will happen.

To see how this works, we'll step through the hull machine until it's complete.
At every iteration, we'll copy the current value of the topology, the list of candidate points that might be on the convex hull, and the queue of edges to inspect.

In [ ]:
from copy import deepcopy
geometries = [deepcopy(hull_machine.geometry)]
candidate_lists = [list(hull_machine.candidates)]
edge_queues = [deepcopy(hull_machine.edge_queue)]

while not hull_machine.is_done():
    hull_machine.step()
    geometries.append(deepcopy(hull_machine.geometry))
    candidate_lists.append(list(hull_machine.candidates))
    edge_queues.append(deepcopy(hull_machine.edge_queue))

Now we'll visualize the state of the algorithm at every step.
Orange points are definitely on the convex hull, as are orange edges.
Blue points might or might not be on the hull and you can see as you step through the algorithm how they become orange once it's obvious that they're inside the current hull.
Blue edges might be on the hull, or they might get split later by the "absorption" of a new hull point.

In [ ]:
from ipywidgets import interact
@interact(step=(0, len(geometries) - 1))
def f(step=0):
    geometry = geometries[step]
    
    fig, ax = plt.subplots()
    ax.set_aspect("equal")

    colors = np.full(len(X), "tab:orange")
    colors[candidate_lists[step]] = "tab:blue"
    zmsh.visualize(geometry, dimension=0, colors=colors, ax=ax)

    colors = [
        "tab:blue" if index in edge_queus[step] else "tab:orange"
        for index in range(len(geometry.topology.cells(1)))
    ]
    zmsh.visualize(geometry, dimension=1, colors=colors, ax=ax)

One of the reasons why we use a first-in, first-out queue to prioritize the edges is because this strategy tends to eliminate the largest number of candidate points the soonest.
The time to execute a single step of the algorithm is proportional to the number of extant candidate points, so by eliminating as many as possible early on, we can keep the total run time lower.

The algorithm is done when either there are no more edges to inspect or there are no more candidate points left.
For this particular problem instance, termination occurred because the candidate points ran out before the edge queue did.

We used a random point set for demonstrative purposes here.
Randomized testing is an extraordinarily useful tool, but computational geometry is full of really dreadful edge cases.
For example, what happens if there are three collinear points on the convex hull of a point set?
The middle point isn't necessary to describe the hull; should we include it or not?
The algorithm we used here does include all points exactly on the hull, including collinear ones.
But generating three collinear points at random using 64-bit floating point arithmetic is so unlikely that it's practically impossible.
So a naive randomized test suite would be unlikely to find this edge case and the test suite for zmsh explicitly checks for it.

Similarly, the acceleration strategy that we use based on eliminating candidate points works well for randomly-distributed inputs.
But if the input points all lie on a circle then this strategy wastes time and the algorithm runs in time $\mathcal{O}(n^2)$.
This edge case won't make the algorithm fail to return a correct result, which collinear points could, so in that sense it's less severe.
You can think of this edge case as being similar to applying quicksort on an already-sorted list.